In [1]:
import altair as alt
import pandas as pd
from pony.orm import *
from datetime import datetime
import random
from bail.db import DB, Case, Inmate

In [2]:
# Connect to SQLite database using Pony
db = DB()

In [3]:
statuses = set(select(i.status for i in Inmate))
pretrial = set(s for s in statuses if ("Prearraignment" in s or "Pretrial" in s))
  
pretrial_inmates = list(select(i for i in Inmate if i.status in pretrial))
other_inmates = list(select(i for i in Inmate if not i.status in pretrial))
  
black_pretrial_inmates = list(select(i for i in Inmate if i.race == "African American" and i.status in pretrial))
white_pretrial_inmates = list(select(i for i in Inmate if i.race == "Caucasian" and i.status in pretrial))
other_pretrial_inmates = list(select(i for i in Inmate if 
                                   i.race != "African American" and 
                                   i.race != "Caucasian" and
                                   i.status in pretrial))

In [4]:
ex = black_pretrial_inmates[0]
list(ex.cases)
ex.case_numbers()

{'20CF002901', '20CM002627', '21CF000085', '21CF000175', '21CF000613'}

In [21]:
def severity(inmate):
    if any("Felony" in x.severity for c in inmate.cases for x in c.charges):
        return 'felony'
    else:
        return 'misdemeanor'
    
def bail_value(inmate):
    bails = [c.cash_bond for c in inmate.cases if c.cash_bond]
    if len(bails) == 0:
        bails = [0]
    return max(bails)

def build_data(inmates):
    return pd.DataFrame([{
        'race': x.race,
        'bail': bail_value(x),
        'days_in_prison': x.days_in_prison(),
        'severity': severity(x),
    } for x in inmates])

data = build_data(pretrial_inmates)

In [32]:
# Prefab stable colors for charts
domain = ['African American', 'American Indian or Alaskan Native', 'Asian or Pacific Islander', 'Caucasian', 'Hispanic', 'Unknown']
range_ = ['red', 'green', 'blue', 'orange', 'purple', 'grey']
race = alt.Color('race:N', scale=alt.Scale(domain=domain, range=range_))

# Facet across both felonies and misdemeanors
alt.Chart(data).mark_circle(size=30, opacity clip=True).encode(
    y=alt.Y('days_in_prison:Q',
        scale=alt.Scale(domain=(0, 365))
    ),
    x=alt.X('bail:Q',
        scale=alt.Scale(domain=(0, 1000))
    ),
    color=race,
).facet(
    row='severity:N'
)

alt.FacetChart(...)

In [31]:
# Misdemeanors only with trend lines
misdemeanors = data.query('severity=="misdemeanor"')

base = alt.Chart(misdemeanors).mark_circle(size=30, clip=True).encode(
    y=alt.Y('days_in_prison:Q',
        scale=alt.Scale(domain=(0, 365))
    ),
    x=alt.X('bail:Q',
        scale=alt.Scale(domain=(0, 800))
    ),
    color=race,
)

base + base.transform_loess('x', 'y', groupby=['race']).mark_line(size=4)

alt.LayerChart(...)